In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re
import nltk
import numpy as np

# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [ ]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [ ]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [3]:
recipes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Notebooks/TOBD/06_string/recipes_sample.csv')

In [5]:
random5 = recipes.sample(5)
max_id = recipes.id.astype(str).str.len().max() + 8
max_minutes = recipes.minutes.astype(str).str.len().max() + 8
def print_f(id_, minutes):
    print(
        f'|{id_:^{max_id}}|{minutes:^{max_minutes}}|'
    )

print_f('id', 'minutes')
print('|'+"-"*(max_id+max_minutes+1)+'|')
for id_, row in random5.iterrows():
    print_f(row.id, row.minutes)

|      id      |   minutes    |
|-----------------------------|
|    354056    |      80      |
|    294826    |      60      |
|    115753    |      60      |
|    311304    |      10      |
|    433106    |      5       |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [6]:
from bs4 import BeautifulSoup
with open('/content/drive/MyDrive/Colab Notebooks/Notebooks/TOBD/06_string/steps_sample.xml') as f:
    ab = BeautifulSoup(f, 'xml')

In [7]:
rcps = ab.find_all('recipe')
rcps = {int(item.find('id').next) : [j.next for j in item.find_all("step")] for item in rcps}

In [8]:
steps = pd.DataFrame(rcps.items(), columns=['id', 'steps'])
steps

,id,steps
0,44123,"[in 1 / 4 cup butter , saute carrots , onion ,..."
1,67664,"[mix all the ingredients using a blender, pour..."
2,38798,[combine all ingredients in a large bowl and m...
3,35173,[lay out sandwich rolls on jelly roll pans / c...
4,84797,[honey mustard sauce: whisk all the ingredient...
...,...,...
29995,267661,"[grease a bread tin very well with butter, min..."
29996,386977,"[in a small bowl , combine the yeast and half ..."
29997,103312,"[for the dough:, dissolve the yeast in the luk..."
29998,486161,"[heat oil in a 4-quart dutch oven, add celery ..."


In [ ]:
def show_info(name, steps, minutes, author_id):
  steps = [item.capitalize() for item in steps]
  
  indexes = []
  for i in range(len(steps)):
    indexes.append(str(i+1))
    
  steps_info = [] 
  for index, step in zip(indexes, steps):
    steps_info.append(index + '. ' + step + '\n')
  steps_info = ''.join(steps_info)

  return f'"{name.title()}"\n\n{steps_info}----------\nАвтор: {author_id}\nСреднее время приготовления: {minutes} минут\n'

print(
    show_info(
        name="george s at the cove black bean soup",
        steps=[
                "clean the leeks and discard the dark green portions",
                "cut the leeks lengthwise then into one-inch pieces",
                "melt the butter in a medium skillet , med",
              ],
        minutes=90,
        author_id=35193
        )
)

"George S At The Cove Black Bean Soup"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
----------
Автор: 35193
Среднее время приготовления: 90 минут



In [ ]:
def show_info2(id):
  recipe = recipes.loc[recipes['id'] == id]
  step = steps.loc[steps['id'] == id].iloc[0, 1]
  
  indexes = []
  for i in range(len(step)):
    indexes.append(str(i+1))
  steps_info = [] 
  for index, step in zip(indexes, step):
    steps_info.append(index + '. ' + step.capitalize() + '\n')
  steps_info = ''.join(steps_info)

  return f'"{recipe.iloc[0, 0].title()}"\n\n{steps_info}----------\nАвтор: {recipe.iloc[0, 3]}\nСреднее время приготовления: {recipe.iloc[0, 2]} минут\n'

print(show_info2(170895))

"Leeks And Parsnips  Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addi

In [ ]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [ ]:
item = steps.loc[steps['id'] == 25082].iloc[0, 1]

['proof yeast in half a cup of the water , with about a teaspoon of flour- let it sit , covered , until it becomes bubbly& active',
 'combine the rest of the water with the yeast / water in a large bowl , and gradually add about half of the flour',
 'stir the mixture about 100 times in the same direction , until it is well-mixed and strands are beginning to form',
 'stir in the salt and oil',
 'add the remaining flour half a cup at a time , stirring well , until it is too stiff to stir with a spoon',
 'turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board',
 'the dough should be springy and nice to work with',
 'when it has been sufficiently kneaded , cover it with a damp cloth for about 10 minutes and wash and grease the bowl lightly',
 'after it has rested , knead the dough a few more times and place it in the bowl , covered with the damp cloth , a plastic bag , and maybe a plate',
 'let the doug

In [ ]:
re.findall(r'\d+ (?:(?:hours?)|(?:minutes?))\b', str(item))

['20 minutes',
 '10 minutes',
 '2 hours',
 '10 minutes',
 '20 minutes',
 '30 minutes']

4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [ ]:
lst = recipes['description'].tolist()
list_indexes = []

for i in lst:
  p = re.compile(r'(^this).*(,\s?but)', re.I|re.X)
  r = p.match(str(i))
  if r:
    list_indexes.append(lst.index(i))

  if lst.index(i) == 29340 or lst.index(i) == 28730 or lst.index(i) ==  28658:
    print(r)

recipes[recipes.index.isin(list_indexes)]

<re.Match object; span=(0, 30), match='this is delicious anytime, but'>
<re.Match object; span=(0, 34), match='this is fabulous for a brunch, but'>
<re.Match object; span=(0, 20), match='this is so good, but'>


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
76,neezy peazy lunch,24760,35,29291,2002-04-08,NaN,"this is a great meal eaten the same day ,but e...",6.0
87,the real black tie mousse cake by olive garden,392181,180,1366440,2009-09-28,NaN,this is it. the real deal. very time consuming...,13.0
92,0 fat chunky watermelon salsa,176996,45,89831,2006-07-10,NaN,this is fantastic served with grilled seafood ...,8.0
183,3 pepper quiche,311313,75,186802,2008-07-02,NaN,this was adapted from a recipe i found on the ...,13.0
202,4 ingredient chocolate nut toffee,148114,25,90628,2005-12-12,9.0,this is the simplest toffee recipe ever. i lo...,4.0
...,...,...,...,...,...,...,...,...
29965,zucchini sausage casserole,472482,80,447199,2012-01-20,13.0,this was a real hit with my family! this recip...,NaN
29967,zucchini spice cake,80067,65,4439,2004-01-02,11.0,"this recipe is pretty close to bizzare, but it...",4.0
29972,zucchini tart with gruyere cheese and herbs,378799,35,177443,2009-06-24,16.0,"this tart is wonderful picnic, potluck or part...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [ ]:
text = 'asdf ada a / b, c / g asdf asdf ,. asdf '
target = '/'
r = r'(\s+)[^\w+\d+](\s+)'.format(target)

s = re.sub(r, '/', text)
print(s)

asdf ada a/b, c/g asdf asdf ,. asdf 


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [9]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package bllip_wsj_no_aux to /root/nltk_data...
       |   Unzipping models/bllip_wsj_no_aux.zip.
       | Downloading package book_grammars to /root/nltk_data...
       |   Unzipping grammars/book_grammars.zip.
       | Downloading 


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [10]:
res = steps.steps.apply(lambda row: [nltk.word_tokenize(step.lower()) for step in row ])
res = {word for row in res for step in row for word in step}

In [12]:
len(res), res

(19564,
 {'boullion',
  'immulsion',
  'hollandaise',
  'tossed',
  'buns-',
  'cylinder',
  'centigrade',
  'poached',
  'requisite',
  'pizzadilla',
  'lightly-greased',
  'minutes-stirring',
  'nutmeg',
  'curve',
  'casserole',
  'powdered-sugar',
  '8x6',
  'pikelets',
  'doom',
  'invest',
  'pancakey',
  'sugar-flour',
  'skim',
  '20-40',
  'narrow-bladed',
  'milk',
  'zippered',
  'mcdonald',
  'inside',
  'garlic-cook',
  'pancake',
  'programme',
  'driving',
  'antennae',
  'combien',
  'job',
  'caramel-nut',
  'basmatic',
  'noisy',
  'tune',
  'damped',
  'applying',
  'decorate',
  'shallots',
  'noting',
  'pla',
  'sausepan',
  '8260',
  'spaetzles',
  'manfacturer',
  'angled',
  'fungus',
  'fineness',
  'thumps',
  'pimiento',
  'nutritious',
  'xture',
  'slabs',
  'noodles',
  '200-degree',
  'window',
  'flaking',
  'trun',
  'temperate',
  'fluids',
  'incredibly',
  'michael',
  'habenaro',
  'cube',
  'french-fried',
  '8x4',
  '290g',
  'cocoa-margarine',
 

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [20]:
recipes['recipe description'] = recipes.description.fillna("").apply(nltk.sent_tokenize)

In [23]:
top_5 = recipes.sort_values(by="recipe description", key= lambda x: x.str.len(), ascending=False).iloc[:5]
top_5

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,recipe description
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN,[this wonderful icing is used for icing cakes ...
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0,[a translucent golden-brown crust allows the g...
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0,[this is one of the best soups i've ever made ...
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN,[the first time i made this cake i grated a mi...
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN,[i wrote this because there are an astounding ...


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [24]:
target = recipes[recipes.id==241106].iloc[0]

In [26]:
def sentence_info(sent):
    array = nltk.pos_tag(nltk.word_tokenize(sent))
    temp = [(max([len(i) for i in pair]),*pair) for pair in array]

    for i in range(2,0,-1):
        for item in temp:
            print(f"{item[i]:^{item[0]}}", end=" ")
        print()

In [28]:
sentence_info(target["name"])

   JJ     NNS    IN     NNS    VBP    JJ   CC   JJ    NNS   
eggplant steaks with chickpeas feta cheese and black olives 
